In [1]:
import numpy as np, pandas as pd
import os,glob, re
%matplotlib inline

dfs = {re.search('([a-zA-Z_]*)\.csv', fn).group(1):pd.read_csv(fn) for fn in glob.glob(os.getcwd()+'//Raw//*.csv')}
print('data frames read:{}'.format(list(dfs.keys())))

print('local variables with the same names are created.')
for k, v in dfs.items(): locals()[k] = v

data frames read:['air_reserve', 'air_store_info', 'air_visit_data', 'date_info', 'hpg_reserve', 'hpg_store_info', 'sample_submission', 'store_id_relation']
local variables with the same names are created.


In [2]:
air_store_info['city'] = [x[0].split(' ')[0] for x in air_store_info['air_area_name'].str.split('-')]
air_store_info['loc'] = [x[1].split(' ')[1] for x in air_store_info['air_area_name'].str.split('-')]

In [3]:
air_visit_data = air_visit_data.merge(air_store_info,on = 'air_store_id',how = 'left')

In [4]:
# air_visit_data['dow'] = pd.to_datetime(air_visit_data['visit_date']).dt.dayofweek
# air_visit_data['woy'] = pd.to_datetime(air_visit_data['visit_date']).dt.weekofyear - 17
# air_visit_data['yr'] = pd.to_datetime(air_visit_data['visit_date']).dt.year
# air_visit_data.tail()

In [5]:
# air_visit_data.groupby(['yr','woy','city'])['air_store_id'].nunique().unstack().plot(figsize = (20,5))

In [6]:
# air_visit_data.groupby(['yr','woy','city'])['visitors'].mean().unstack().plot(figsize = (20,5))

In [7]:
print("Add features to date_info")
print()
print("1. Remove Saturday and Sunday as flagged holidays")
weekdayholidays = date_info.apply(lambda x: x.day_of_week in ['Saturday','Sunday'] and x.holiday_flg == 1,axis=1)
date_info.loc[weekdayholidays,'holiday_flg'] = 0
print("2. Add day before the holiday feature")
date_info['lagged'] = date_info['holiday_flg'].shift(-1)
day_b4_holiday = (date_info['holiday_flg'] == 0) & (date_info['lagged'] == 1.0) #& (date_info['day_of_week'] not ['Sunday','Saturday'])
date_info['dayb4holiday'] = 0
date_info.loc[day_b4_holiday,'dayb4holiday'] = 1
date_info.drop('lagged',axis = 1, inplace = True)
print("3. Add year, month, day of year features")
date_info['year'] = pd.to_datetime(date_info['calendar_date']).dt.year
date_info['month'] = pd.to_datetime(date_info['calendar_date']).dt.month
date_info['woy'] = pd.to_datetime(date_info['calendar_date']).dt.weekofyear
#date_info['doy'] = pd.to_datetime(date_info['calendar_date']).dt.dayofyear
date_info.head()

Add features to date_info

1. Remove Saturday and Sunday as flagged holidays
2. Add day before the holiday feature
3. Add year, month, day of year features


,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy
0,2016-01-01,Friday,1,0,2016,1,53
1,2016-01-02,Saturday,0,0,2016,1,53
2,2016-01-03,Sunday,0,0,2016,1,53
3,2016-01-04,Monday,0,0,2016,1,1
4,2016-01-05,Tuesday,0,0,2016,1,1


In [8]:
goldenweek_2016start = '2016-04-29' 
goldenweek_2016end = '2016-05-05'
goldenweek_2017start = '2017-04-29' 
goldenweek_2017end = '2017-05-07'

date_info['golden'] = 0
date_info.loc[((date_info['calendar_date'] >= goldenweek_2016start) & (date_info['calendar_date'] <= goldenweek_2016end))
              | ((date_info['calendar_date'] >= goldenweek_2017start) & (date_info['calendar_date'] <= goldenweek_2017end))
              ,'golden'] = 1
date_info[date_info['golden'] == 1]

,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,golden
119,2016-04-29,Friday,1,0,2016,4,17,1
120,2016-04-30,Saturday,0,0,2016,4,17,1
121,2016-05-01,Sunday,0,0,2016,5,17,1
122,2016-05-02,Monday,0,1,2016,5,18,1
123,2016-05-03,Tuesday,1,0,2016,5,18,1
124,2016-05-04,Wednesday,1,0,2016,5,18,1
125,2016-05-05,Thursday,1,0,2016,5,18,1
484,2017-04-29,Saturday,0,0,2017,4,17,1
485,2017-04-30,Sunday,0,0,2017,4,17,1
486,2017-05-01,Monday,0,0,2017,5,18,1


In [9]:
date_info.loc[date_info['golden'] == 1,'holiday_flg'] = 0
date_info.loc[date_info['golden'] == 1,'dayb4holiday'] = 0
date_info[date_info['golden'] == 1]

,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,golden
119,2016-04-29,Friday,0,0,2016,4,17,1
120,2016-04-30,Saturday,0,0,2016,4,17,1
121,2016-05-01,Sunday,0,0,2016,5,17,1
122,2016-05-02,Monday,0,0,2016,5,18,1
123,2016-05-03,Tuesday,0,0,2016,5,18,1
124,2016-05-04,Wednesday,0,0,2016,5,18,1
125,2016-05-05,Thursday,0,0,2016,5,18,1
484,2017-04-29,Saturday,0,0,2017,4,17,1
485,2017-04-30,Sunday,0,0,2017,4,17,1
486,2017-05-01,Monday,0,0,2017,5,18,1


In [10]:
date_info['weights'] = ((date_info.index + 1)/ len(date_info))**7
date_info.tail()

,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,golden,weights
512,2017-05-27,Saturday,0,0,2017,5,21,0,0.947082
513,2017-05-28,Sunday,0,0,2017,5,21,0,0.960081
514,2017-05-29,Monday,0,0,2017,5,22,0,0.973233
515,2017-05-30,Tuesday,0,0,2017,5,22,0,0.986539
516,2017-05-31,Wednesday,0,0,2017,5,22,0,1.000000


In [11]:
air_visit_data = air_visit_data.merge(date_info,left_on = 'visit_date',right_on='calendar_date',how= 'left')
# air_visit_data.drop(['visit_date','calendar_date','air_area_name'],axis = 1,inplace = True)
air_visit_data.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,city,loc,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,golden,weights
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-13,Wednesday,0,0,2016,1,2,0,6.355792e-12
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-14,Thursday,0,0,2016,1,2,0,1.067732e-11
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-15,Friday,0,0,2016,1,2,0,1.730633e-11
3,air_ba937bf13d40fb24,2016-01-16,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-16,Saturday,0,0,2016,1,2,0,2.718980e-11
4,air_ba937bf13d40fb24,2016-01-18,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-18,Monday,0,0,2016,1,3,0,6.201171e-11


In [12]:
air_reserve['visit_date'] = pd.to_datetime(air_reserve['visit_datetime']).dt.date
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01


In [13]:
air_reserve = air_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors'].sum().reset_index()
air_reserve.head()

,air_store_id,visit_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-05,9
2,air_00a91d42b08b08d9,2016-12-14,18
3,air_00a91d42b08b08d9,2016-12-17,2
4,air_00a91d42b08b08d9,2016-12-20,4


In [14]:
air_visit_data = air_visit_data.merge(air_reserve, on = ['air_store_id','visit_date'],how = 'left').fillna(-1)

In [15]:
air_visit_data.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,city,loc,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,golden,weights,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-13,Wednesday,0,0,2016,1,2,0,6.355792e-12,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-14,Thursday,0,0,2016,1,2,0,1.067732e-11,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-15,Friday,0,0,2016,1,2,0,1.730633e-11,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-16,Saturday,0,0,2016,1,2,0,2.718980e-11,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,2016-01-18,Monday,0,0,2016,1,3,0,6.201171e-11,-1.0


In [16]:
air_visit_data.drop(['calendar_date','visit_date'],axis = 1,inplace = True)

In [ ]:
air_visit_data.drop('air_area_name',axis =1,inplace = True)
sample_submission.drop('air_area_name',axis =1,inplace = True)

In [ ]:
print("Split id column in sample_submission")
sample_submission["air_store_id"],sample_submission["visit_date"] = sample_submission.id.str[:20],sample_submission.id.str[21:]
sample_submission.head()

sample_submission = sample_submission.merge(date_info,left_on='visit_date',right_on = 'calendar_date',how = 'left')

sample_submission = sample_submission.merge(air_reserve,on=['air_store_id','visit_date'],how = 'left')

sample_submission = sample_submission.merge(air_store_info, on ='air_store_id',how = 'left')

sample_submission.drop(['calendar_date','visit_date'],axis = 1,inplace = True)

sample_submission.head()

In [ ]:
print("Encode all categorical labels")

le = LabelEncoder()

air_visit_data['air_store_id'] = le.fit_transform(air_visit_data['air_store_id'])
sample_submission['air_store_id'] = le.transform(sample_submission['air_store_id'])

# air_store_info['air_store_id'] = le.transform(air_store_info['air_store_id'])
# air_reserve['air_store_id'] = le.transform(air_reserve['air_store_id'])
# air_visit_data['air_store_id'] = le.transform(air_visit_data['air_store_id'])
# hpg_reserve['air_store_id'] = le.transform(hpg_reserve['air_store_id'])
# hpg_store_info['air_store_id'] = le.transform(hpg_store_info['air_store_id'])
# sample_submission['air_store_id'] = le.transform(sample_submission['air_store_id'])

le.fit(air_store_info['air_genre_name'])
air_store_info['air_genre_name'] = le.fit_transform(air_store_info['air_genre_name'])
sample_submission['air_genre_name'] = le.transform(sample_submission['air_genre_name'])

# le.fit(hpg_store_info['hpg_genre_name'])
# hpg_store_info['hpg_genre_name'] = le.fit_transform(hpg_store_info['hpg_genre_name'])

le.fit(air_store_info['air_area_name'])
air_store_info['air_area_name'] = le.fit_transform(air_store_info['air_area_name'])
sample_submission['air_area_name'] = le.transform(sample_submission['air_area_name'])

# le.fit(hpg_store_info['hpg_area_name'])
# hpg_store_info['hpg_area_name'] = le.fit_transform(hpg_store_info['hpg_area_name'])

# le.fit(date_info['day_of_week'])
# date_info['day_of_week'] = le.fit_transform(date_info['day_of_week'])

# print("Encoding completed")

In [ ]:
print("import required modules")
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

from ml_metrics import rmsle

In [18]:
air_visit_data.head()

,air_store_id,visitors,air_genre_name,air_area_name,latitude,longitude,city,loc,day_of_week,holiday_flg,dayb4holiday,year,month,woy,golden,weights,reserve_visitors
0,air_ba937bf13d40fb24,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Wednesday,0,0,2016,1,2,0,6.355792e-12,-1.0
1,air_ba937bf13d40fb24,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Thursday,0,0,2016,1,2,0,1.067732e-11,-1.0
2,air_ba937bf13d40fb24,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Friday,0,0,2016,1,2,0,1.730633e-11,-1.0
3,air_ba937bf13d40fb24,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Saturday,0,0,2016,1,2,0,2.718980e-11,-1.0
4,air_ba937bf13d40fb24,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Monday,0,0,2016,1,3,0,6.201171e-11,-1.0


Split id column in sample_submission


,id,visitors,air_store_id,day_of_week,holiday_flg,dayb4holiday,year,month,woy,golden,weights,reserve_visitors,air_genre_name,air_area_name,latitude,longitude,city,loc
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,Sunday,0,0,2017,4,16,0,0.586023,NaN,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Tōkyō,Chiyoda
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,Monday,0,0,2017,4,17,0,0.594641,NaN,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Tōkyō,Chiyoda
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,Tuesday,0,0,2017,4,17,0,0.603367,NaN,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Tōkyō,Chiyoda
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,Wednesday,0,0,2017,4,17,0,0.612203,NaN,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Tōkyō,Chiyoda
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,Thursday,0,0,2017,4,17,0,0.621149,NaN,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,Tōkyō,Chiyoda
